In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import drizzlepac
from stsci.tools import teal
import os

The following task in the stsci.skypac package can be run with TEAL:
                                    skymatch                                    
The following tasks in the drizzlepac package can be run with TEAL:
    astrodrizzle       config_testbed      imagefindpars           mapreg       
       photeq            pixreplace           pixtopix            pixtosky      
  refimagefindpars       resetbits          runastrodriz          skytopix      
     tweakback            tweakreg           updatenpol


In [ ]:
from astropy.table import Table
from astroquery.mast import Observations
from stwcs.wcsutil import headerlet
from stwcs import updatewcs

In [5]:
os.chdir('/Users/orion/phd_research/hst/1025+390/f336w')

In [ ]:
filename = 'idfq06e7q_flc.fits'       #raw target image
fits.info(filename)

ext_indices = headerlet.find_headerlet_HDUs(filename, strict=False)
#print(ext_indices)

def get_hdrlet_wcsnames(filename):
    """Print and return list of WCS names in HDRLET extensions of fits file"""
    hdu = fits.open(filename)
    ext_indices = headerlet.find_headerlet_HDUs(filename, strict=False)

    print('Ext\tWCSNAME')
    new_wcsnames = []
    for ext_ind in ext_indices:
        name = hdu[ext_ind].header['WCSNAME']
        print(ext_ind, '\t', name)
        new_wcsnames.append(name)
        
    hdu.close()
    return new_wcsnames

new_wcsnames = get_hdrlet_wcsnames(filename)

#WCS solution currently in use
current_wcs = fits.getval(filename, 'WCSNAME', ext=('SCI', 1))
print(current_wcs)

In [ ]:
#refcat = '/home/chetna/Documents/hst_new/1025+390/gaia_1025+390.cat'
#wcsname = 'Gaia'                       # Specify the WCS name for this alignment
#refcat = '/Users/orion/phd_research/hst2/sdss_cat_lists/sdss_1128+455.cat'
#wcsname='sdss'

from drizzlepac import tweakreg
teal.unlearn('tweakreg')
tweakreg.TweakReg('*flc.fits',
                  minobj=5,             # min. objects for matching images (default-15)
                  searchrad=2.0,       # default = 1.0 arcsec
                  updatehdr=True,      # switch update-header to 'TRUE' only on the last run
                  conv_width=3.5, 
                  threshold=200, 
                  peakmax=10000, 
                  #refcat=refcat,       # User supplied catalog (Gaia)
                  #shiftfile=True,      # Save output shift file (so we can look at shifts later)
                  #outshifts='sdss_shifts.txt',
                  #wcsname=wcsname,     # Give WCS a new name
                  reusename=True,
                  clean=True)


Setting up logfile :  tweakreg.log
TweakReg Version 1.4.7(18-April-2018) started at: 04:19:32.781 (05/07/2020) 

Version Information
--------------------
Python Version [GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
3.6.10 |Anaconda, Inc.| (default, May  7 2020, 23:06:31) 
numpy Version -> 1.17.4 
astropy Version -> 4.0.1.post1 
stwcs Version -> 1.5.3 

Finding shifts for: 
    idfq05eaq_flc.fits
    idfq05elq_flc.fits
    idfq05edq_flc.fits
    idfq05ebq_flc.fits
    idfq05epq_flc.fits
    idfq05enq_flc.fits

===  Source finding for image 'idfq05eaq_flc.fits':
  #  Source finding for 'idfq05eaq_flc.fits', EXT=('SCI', 1) started at: 04:19:33.45 (05/07/2020)
     Found 19797 objects.
  #  Source finding for 'idfq05eaq_flc.fits', EXT=('SCI', 2) started at: 04:19:36.060 (05/07/2020)
     Found 19440 objects.
===  FINAL number of objects in image 'idfq05eaq_flc.fits': 39237

===  Source finding for image 'idfq05elq_flc.fits':
  #  Source finding for 'idfq05elq_flc.fits', EXT=(

In [ ]:
#for line in open('Gaia_shifts.txt').readlines():
for line in open('sdss_shifts.txt').readlines():
    print(line)

In [ ]:
#choosing the headerlet to use -> doesn't work with Tweakreg
hdrlet_hdrnames = headerlet.get_headerlet_kw_names(fits.open(filename), 'HDRNAME')
desired_hdrname = hdrlet_hdrnames[new_wcsnames.index('IDC_2731450pi')]

print(desired_hdrname)
headerlet.restore_from_headerlet(filename, hdrname=desired_hdrname, archive=False, force=False)

In [4]:
from drizzlepac import astrodrizzle
teal.unlearn('astrodrizzle')
astrodrizzle.AstroDrizzle('*flc.fits',
                          output='f225w',
                          build=True,
                          preserve=False,
                          context=False,
                          skystat='median',
                          final_scale=0.1,
                          final_pixfrac=1,
                          driz_sep_bits='64,32',
                          driz_cr_corr=True,
                          driz_cr_grow=3,
                          driz_cr_snr='4.0 3.0',
                          final_wht_type='EXP',
                          clean=True)

Setting up logfile :  astrodrizzle.log
AstroDrizzle Version 3.1.6 (2020-02-19 16:16:30 -0500) started at: 04:11:33.109 (05/07/2020)

==== Processing Step  Initialization  started at  04:11:33.116 (05/07/2020)

WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 45.43901032411608  35.200470661378695  
CRPIX : 817.5682179108262  869.0742190405726  
CD1_1 CD1_2  : -1.3030989794505569e-05  -2.4531576452545745e-05  
CD2_1 CD2_2  : -2.4531576452545745e-05  1.3030989794505569e-05  
NAXIS : 1634  1737
********************************************************************************
*
*  Estimated memory usage:  up to 386 Mb.
*  Output image size:       1634 X 1737 pixels. 
*  Output image file:       ~ 32 Mb. 
*  Cores available:         4
*
********************************************************************************
==== Processing Step Initialization finished at 04:11:35.104 (05/07/2020)
==== Processing Step  Static Mask  started at  04:11:35.126 (05/07/2020)

===

CD2_1 CD2_2  : -2.4531576452545745e-05  1.3030989794505569e-05  
NAXIS : 1634  1737
-Generating multi-extension output file:  f225w_drc.fits
Deleted all instances of WCS with key A in extensions [1]
Deleted all instances of WCS with key B in extensions [1]
Writing out to disk: f225w_drc.fits
==== Processing Step Final Drizzle finished at 04:12:42.950 (05/07/2020)

AstroDrizzle Version 3.1.6 is finished processing at 04:12:42.952 (05/07/2020).



   --------------------          --------------------
                   Step          Elapsed time
   --------------------          --------------------

         Initialization          1.9878 sec.
            Static Mask          1.7700 sec.
           Subtract Sky          4.1151 sec.
       Separate Drizzle          6.2308 sec.
          Create Median          9.0041 sec.
                   Blot          12.1318 sec.
                Driz_CR          9.4373 sec.
          Final Drizzle          25.1030 sec.
   ====================          

In [ ]:
from drizzlepac import tweakreg
teal.unlearn('tweakreg')
tweakreg.TweakReg('*drc.fits',
                  refimage='panstarrs.fits',
                  minobj=1,             # min. objects for matching images (default-15)
                  searchrad=60.0,       # default = 1.0 arcsec
                  updatehdr=False,      # switch update-header to 'TRUE' only on the last run
                  conv_width=3.5, 
                  threshold=2, 
                  peakmax=50000, 
                  #refcat=refcat,       # User supplied catalog (Gaia)
                  #shiftfile=True,      # Save out shift file (so we can look at shifts later)
                  #outshifts='Gaia_shifts.txt',
                  #wcsname=wcsname,     # Give WCS a new name
                  #reusename=True,
                  clean=True)